# **Visualization** - Heatmaps:

The point of this notebook, the final notebook in the series, will be to create the end product of heatmpas related to the different locational granularities. Here I won't be describing every step as most of it is formating data and columns to the excepted orginizations for the plotly express choropleth_mapbox or density heatmpas below. Enjoy the beautiful animations as a reward for getting this far, I know I will.<br>
-- Connell

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import pickle
import json
from IPython.display import display, HTML
import streamlit.components.v1 as components


In [ ]:
# Paths to your files
precincts_geojson_path = r"C:\Users\cmphi\Documents\github\Capstone_911\data\Police Precincts\Police Precincts.geojson"
boroughs_geojson_path = r"C:\Users\cmphi\Documents\github\Capstone_911\data\Borough Boundaries\Borough Boundaries.geojson"
data_dict_path = r"C:\Users\cmphi\Documents\github\Capstone_911\data\forecast_3-31-2024.pkl"

# Load the GeoJSON files
boroughs_geo = gpd.read_file(boroughs_geojson_path)
precincts_geo = gpd.read_file(precincts_geojson_path)

# Load the data dictionary
data_dict = pd.read_pickle(data_dict_path)

In [ ]:
boroughs_geo.head()

In [ ]:
boroughs_geo['boro_name'] = boroughs_geo['boro_name'].str.upper()
boroughs_geo.head()

In [ ]:
precincts_geo.head()

In [ ]:
# Shortening dataframe 1-test, 2-1predictions, 3-12predictions, 4-year
length = 1
year_length = 24*365

match length:
    case 1:
        for key, df in data_dict.items():
            if key != 'scores':
                data_dict[key] = df.tail(24)

    case 2:
        for key, df in data_dict.items():
            if key != "scores":
                data_dict[key] = df.loc[df['quality'] == 1,:]

    case 3:
        for key, df in data_dict.items():
            if key != 'socres':
                data_dict[key] = df.loc[df['quality'].isin([1, 2]), :]

    case 4:
        for key, df in data_dict.items():
            if key != 'scores':
                data_dict[key] = df.tail(year_length)


data_dict

In [ ]:
# Exclude specific keys
excluded_keys = ['scores', 'total']

# Filter and add a source column based on key type (numeric as strings) and excluding specific keys
precinct_data = {key: df.assign(Source=key) for key, df in data_dict.items() if key.isdigit() and key not in excluded_keys}
borough_data = {key: df.assign(Source=key) for key, df in data_dict.items() if not key.isdigit() and key not in excluded_keys}

# Merge the DataFrames in each category if they are not empty
merged_precinct_df = pd.concat(precinct_data.values(), ignore_index=True) if precinct_data else pd.DataFrame()
merged_borough_df = pd.concat(borough_data.values(), ignore_index=True) if borough_data else pd.DataFrame()

print("Precinct Sources DataFrame:")
display(merged_precinct_df)
print("Borough Sources DataFrame:")
display(merged_borough_df)

Shape files can be found for download at the links below:
* [Borough Boundaries](https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm)
* [Precinct Boundaries](https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz)

The next step below is to make the file a geojson dict so the graph understands the shape file.

In [ ]:
#JSON Conversion
# Convert the GeoPandas DataFrame to a GeoJSON string
geojson_str_boroughs = boroughs_geo.to_json()
geojson_str_precincts = precincts_geo.to_json()
# Optionally, convert the GeoJSON string to a dictionary
geojson_dict_boroughs = json.loads(geojson_str_boroughs)
geojson_dict_precincts = json.loads(geojson_str_precincts)

In [ ]:
# Calculate the minimum and maximum values of 'yhat' across all time frames
min_yhat = merged_borough_df['yhat'].min()
max_yhat = merged_borough_df['yhat'].max()

# Finally, The Plotting
fig_borough = px.choropleth_mapbox(
    merged_borough_df,
    geojson=geojson_dict_boroughs,
    locations='Source',
    color='yhat',
    animation_frame=merged_borough_df['ds'],
    featureidkey='properties.boro_name',
    color_continuous_scale='Turbo', #Bluered
    range_color=[min_yhat, max_yhat],  # Set the range for the color scale
    mapbox_style='carto-darkmatter',  # Free style from OpenStreetMap
    zoom=9.5,  # Adjust the zoom level based on your map
    center={"lat": 40.730610, "lon": -73.935242}  # Center the map around New York City
)

fig_borough.update_geos(fitbounds="locations", visible=False)

# Update layout with faster animation speed
fig_borough.update_layout(
    width=1250,
    height=800,
    margin=dict(l=1, r=0, t=1, b=0),  # Adjust margins to accommodate buttons
    updatemenus=[{
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 250, "redraw": True}, "fromcurrent": True}],  # Adjust duration to make it faster
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }]
)

fig_borough.show()
fig_borough.write_html("../misc/borough_heatmap.html")

In [ ]:
# Calculate the minimum and maximum values of 'yhat' across all time frames
min_yhat = merged_precinct_df['yhat'].min()
max_yhat = merged_precinct_df['yhat'].max()

# Finally, The Plotting
fig_precinct = px.choropleth_mapbox(
    merged_precinct_df,
    geojson=geojson_dict_precincts,
    locations='Source',
    color='yhat',
    animation_frame=merged_precinct_df['ds'],
    featureidkey='properties.precinct',
    color_continuous_scale='Turbo',
    range_color=[min_yhat, max_yhat],  # Set the range for the color scale
    mapbox_style='carto-darkmatter',  # Free style from OpenStreetMap
    zoom=9.5,  # Adjust the zoom level based on your map
    center={"lat": 40.730610, "lon": -73.935242}  # Center the map around New York City
)

fig_precinct.update_geos(fitbounds="locations", visible=False)

fig_precinct.update_layout(
    #autosize = True,
    width=1250,
    height=800,
    margin=dict(l=1, r=0, t=1, b=0),  # Adjust margins to accommodate buttons
)

# Update layout with faster animation speed
fig_precinct.update_layout(
    width=1250,
    height=800,
    margin=dict(l=1, r=0, t=1, b=0),  # Adjust margins to accommodate buttons
    updatemenus=[{
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 250, "redraw": True}, "fromcurrent": True}],  # Adjust duration to make it faster
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }]
)

fig_precinct.show()
fig_precinct.write_html("../misc/precinct_heatmap.html")

```python
#exsample to run .html
with open("../misc/borough_heatmap.html", "r", encoding="utf-8") as f:
    html_content_borough = f.read()

# Embed the HTML in the STREAMLIT APP
components.html(html_content_borough, width=1250, height=800)
```